## Importación de librerías y declaración de funciones

In [7]:
import diseño_registro
from lector_archivo import ArchivoPresentacion
from helpers import listado
import math

def dbl_con_ceros(num, cant):
    return "{:.2f}".format(num).replace('.','').zfill(cant)

## Apertura del TXT

In [8]:
txt = ArchivoPresentacion(r"txt_presentaciones\A10_AUS_A16.txt")

## Modificación de datos dentro del TXT

### Modificación del código de liquidación para ciertos CUILs:

In [10]:
cuils_modificar = listado("""
20276914155
20299391583
20264959013
20369619706
20320995680
20237969341
20355259839
20279576463
20254068056
23315494699
23205421769
20289467220
20181290405

""")

# Respetar el ancho del string
codigo_anterior = 'VAC9      '
codigo_nuevo = '/109A     '

In [11]:
for e in txt.empleados:
    if e in cuils_modificar: # Si es para todos, esta comprobación no se haría.
        for concepto in e.conceptos:
            if concepto['codigo_concepto_liquidado'] == codigo_anterior:
                concepto['codigo_concepto_liquidado'] = codigo_nuevo
                concepto['cantidad'] = '00000'
                concepto['unidades'] = ' '

In [6]:
for e in txt.empleados:
    if e in cuils_modificar:
        e.registro2['dias_prop_tope'] = '030'

### Verificación de que la remuneracion 10 no sea menor al MNI:

In [5]:
MINIMO_NO_IMPONIBLE = 14_601.14

In [ ]:
for e in txt.empleados:
    base10 = int(e.registro4['base_imponible10'])/100.0
    
    if base10 < MINIMO_NO_IMPONIBLE:

        print(f"CUIL: {e}")
        print("La base_imponible10 es menor al mínimo no imponible.")
        
        base8 = int(e.registro4['base_imponible8'])/100.0
        detraccion = max(0,round(base10 - base8,2))
        
        e.registro4['base_imponible10'] = dbl_con_ceros(MINIMO_NO_IMPONIBLE, diseño_registro.DISEÑO4['base_imponible10'])
        e.registro4['importe_detraer'] = dbl_con_ceros(detraccion, diseño_registro.DISEÑO4['importe_detraer'])
        
        print("Se actualizaron los siguientes valores:")
        print(f" base_imponible10: {e.registro4['base_imponible10']}")
        print(f" importe_detraer: {e.registro4['importe_detraer']}")

### Incorporación del SAC a determinados CUILs

In [ ]:
# Pegar a continuación:
#  CUIT Cnt.Dias Importe
info = """
27257004290	120	100,5
20290290393	120	100,5

"""
info = info.strip().split('\n')

cuils_agregar = {}
for i in info:
    cortado = i.split('\t')
    cuils_agregar[cortado[0]] = {'dias': cortado[1], 'importe': float(cortado[2].replace(',','.'))}

cuils_agregar

In [8]:
for e in txt.empleados:
    if e in list(cuils_agregar.keys()):

        dias = str(cuils_agregar[e.cuit]['dias']).zfill(5)
        importe = dbl_con_ceros(cuils_agregar[e.cuit]['importe'], 15)

        e.conceptos.append({
            'registro':                     '03',
            'cuil':                         e.cuit,
            'codigo_concepto_liquidado':    '/S13      ',
            'cantidad':                     dias,
            'unidades':                     'D',
            'importe':                      importe,
            'debcre':                       'C',
            'periodo_retro':                '      '
        })

### Asignación de seguro de vida a Pasantes

In [4]:
for e in txt.empleados:
    if e.registro4['modalidad_contratacion'] == '027':
        e.registro4['marca_scvo'] = '1'

### Eliminar registros de determinados CUILs

In [9]:
cuil_eliminar = listado("""
27408691201
20395468457

""")

In [ ]:
for e in txt.empleados:
    if e in cuil_eliminar:
        txt.empleados.remove(e)

### Adecuación `dias-tope` según remuneraciones sujetas a tope

In [13]:
TOPE_MENSUAL = 474_530.27
TOPE_DIARIO = TOPE_MENSUAL / 30

print('CUIL', 'Rem1', 'Rem4', 'Rem5', 'dias_orig', 'dias_modif')
for e in txt.empleados:

    # Remuneraciones con tope: 1, 4 y 5
    rem1 = int(e.registro4['base_imponible1'])/100.0
    rem4 = int(e.registro4['base_imponible4'])/100.0
    rem5 = int(e.registro4['base_imponible5'])/100.0
    dias = int(e.registro2['dias_prop_tope'])

    tope_aplicable = TOPE_DIARIO * dias
    control = max(rem1, rem4, rem5) > tope_aplicable

    if control:
        
        dias_suficiente = math.ceil(max(rem1, rem4, rem5) / TOPE_DIARIO)
        dias_suficiente = str(dias_suficiente).zfill(3)

        e.registro2['dias_prop_tope'] = dias_suficiente
        print(e, rem1, rem4, rem5, dias, dias_suficiente) 

CUIL Rem1 Rem4 Rem5 dias_orig dias_modif


### Reemplazo tope para `rem1`, `rem4` y `rem5` según tope aplicable

In [ ]:
cuil_modificar = listado("""
27408691201
20395468457

""")

In [ ]:
TOPE_MENSUAL = 474_530.27
TOPE_DIARIO = TOPE_MENSUAL / 30

print('CUIL', 'Rem1', 'Rem4', 'Rem5', 'dias_orig', 'dias_modif', 'nuevo_tope')

for e in txt.empleados:

    # Remuneraciones con tope: 1, 4 y 5
    rem1 = int(e.registro4['base_imponible1'])/100.0
    rem4 = int(e.registro4['base_imponible4'])/100.0
    rem5 = int(e.registro4['base_imponible5'])/100.0
    dias = int(e.registro2['dias_prop_tope'])

    tope_aplicable = TOPE_DIARIO * dias
    control = max(rem1, rem4, rem5) > tope_aplicable

    if control and e in cuil_modificar:
        
        # Modificación de dias para tope
        dias_suficiente = math.ceil(max(rem1, rem4, rem5) / TOPE_DIARIO)
        dias_suficiente = str(dias_suficiente).zfill(3)

        e.registro2['dias_prop_tope'] = dias_suficiente

        # Modificación de bases imponibles
        nuevo_valor = dbl_con_ceros(tope_aplicable, 15)

        e.registro4['base_imponible1'] = nuevo_valor
        e.registro4['base_imponible4'] = nuevo_valor
        e.registro4['base_imponible5'] = nuevo_valor
        
        print(e, rem1, rem4, rem5, dias, dias_suficiente, nuevo_valor) 

## Guardado del archivo modificado

In [ ]:
#txt.guardar(inplace=True)
txt.guardar()